## Setup

In [1]:
import pandas as pd
import numpy as np
import os
from langchain.document_loaders import PyPDFLoader, UnstructuredPDFLoader, PyPDFium2Loader
from langchain.document_loaders import PyPDFDirectoryLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path
import random

## Input data directory
data_dir = "cureus"
inputdirectory = Path(f"./data_input/{data_dir}")
## This is where the output csv files will be written
out_dir = data_dir
outputdirectory = Path(f"./data_output/{out_dir}")

## Load Documents

In [2]:
## Dir PDF Loader
# loader = PyPDFDirectoryLoader(inputdirectory)
## File Loader
# loader = PyPDFLoader("C:\\Users\\go2za\\Documents\\knowledge_graph\\data_input\\cureus\\C2024C00104.pdf")
# loader = DirectoryLoader(inputdirectory, show_progress=True)
documents = loader.load()

splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=150,
    length_function=len,
    is_separator_regex=False,
)

pages = splitter.split_documents(documents)
print("Number of chunks = ", len(pages))
print(pages[3].page_content)


Number of chunks =  398
Renewable Energy (Electricity) Act 2000 i
Compilation No. 30 Compilation date: 20/03/2024 Registered: 02/04/2024Contents
Part 1—Preliminary 1
1 Short title............................................................................................. 1
2 Commencement .................................................................................. 1
3 Object/outline...................................................................................... 1
4 Years to which this Act applies .......................................................... 2
5 Definitions .......................................................................................... 2
6 Act binds Crown ............................................................................... 12
7 Application to the external Territories.............................................. 12
7A Tax deductibility............................................................................... 12
7B Constitutional basis

In [2]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

# Function to load text files from a directory
def load_txt_documents(directory_path):
    documents = []
    for filename in os.listdir(directory_path):
        if filename.endswith('.txt'):
            file_path = os.path.join(directory_path, filename)
            loader = TextLoader(file_path, encoding='ISO-8859-1')
            documents.extend(loader.load())
    return documents

# Load text documents
directory_path = "C:\\Users\\go2za\\Downloads\\AliAustralia\\Stakeholder"
documents = load_txt_documents(directory_path)

# Split the loaded documents into chunks
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=150,
    length_function=len,
    is_separator_regex=False,
)

pages = splitter.split_documents(documents)

# Output the number of chunks and display a specific chunk
print("Number of chunks = ", len(pages))
print(pages[0].page_content)


Number of chunks =  31
This $30 billion project plans to power Singapore with Australian solar - create digital


In [9]:
type(txt_files_contents)

dict

## Create a dataframe of all the chunks

In [6]:
pip install yachalk

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
from helpers.df_helpers import documents2Dataframe
df = documents2Dataframe(pages)
print(df.shape)
df.head()

(31, 3)


text  \
0  This $30 billion project plans to power Singap...   
1  The multi-gigawatt Australia-Asia PowerLink (A...   
2  harness Australiaâs abundant solar resources...   
3  âAny time you are doing something for the fi...   
4  Colin Watson MIEAust\n\n\n\nâThe physical si...   

                                              source  \
0  C:\Users\go2za\Downloads\AliAustralia\Stakehol...   
1  C:\Users\go2za\Downloads\AliAustralia\Stakehol...   
2  C:\Users\go2za\Downloads\AliAustralia\Stakehol...   
3  C:\Users\go2za\Downloads\AliAustralia\Stakehol...   
4  C:\Users\go2za\Downloads\AliAustralia\Stakehol...   

                           chunk_id  
0  c6511989c2c049ab9fe8a53e294bd601  
1  da5cade0de7f4a43aa1a1e05ae6eb9a4  
2  0d63f158b0494adf8cc02cf1363a40c9  
3  dafddc7a30104d1ab3373f001722b21c  
4  0207d441410f4ebf8281b4fde9378f84

## Extract Concepts

In [7]:
## This function uses the helpers/prompt function to extract concepts from text
from helpers.df_helpers import df2Graph
from helpers.df_helpers import graph2Df

If regenerate is set to True then the dataframes are regenerated and Both the dataframes are written in the csv format so we dont have to calculate them again. 

        dfne = dataframe of edges

        df = dataframe of chunks


Else the dataframes are read from the output directory

In [ ]:
# To regenerate the graph with LLM, set this to True
regenerate = True

if regenerate:
    concepts_list = df2Graph(df, model='zephyr:latest')
    dfg1 = graph2Df(concepts_list)
    if not os.path.exists(outputdirectory):
        os.makedirs(outputdirectory)
    
    dfg1.to_csv(outputdirectory/"graph.csv", sep="|", index=False)
    df.to_csv(outputdirectory/"chunks.csv", sep="|", index=False)
else:
    dfg1 = pd.read_csv(outputdirectory/"graph.csv", sep="|")

dfg1.replace("", np.nan, inplace=True)
dfg1.dropna(subset=["node_1", "node_2", 'edge'], inplace=True)
dfg1['count'] = 4 

# Increasing the weight of the relation to 4. 
# We will assign the weight of 1 when later the contextual proximity will be calculated.  
print(dfg1.shape)
dfg1.head()

[
   {
       "node_1": "$30 billion",
       "node_2": "project",
       "edge": "The $30 billion project is a planned endeavor."
   },
   {
       "node_1": "Singapore",
       "node_2": "power",
       "edge": "The $30 billion project plans to provide power to Singapore."
   },
   {
       "node_1": "Australian solar",
       "node_2": "project",
       "edge": "The $30 billion project involves using solar energy from Australia."
   },
   {
       "node_1": "digital",
       "node_2": "create",
       "edge": "This $30 billion project also aims to generate digital capabilities."
   }
][
   {
       "node_1": "Australia-Asia PowerLink (AAPowerLink)",
       "node_2": "renewable electricity",
       "edge": "The multi-gigawatt Australia-Asia PowerLink (AAPowerLink) is a high-capacity solar generation, storage, and transmission system that will send large volumes of renewable electricity from the Northern Territory to markets in Darwin and Singapore."
   }, {
       "node_1": "Australi

## Calculating contextual proximity

In [ ]:
def contextual_proximity(df: pd.DataFrame) -> pd.DataFrame:
    ## Melt the dataframe into a list of nodes
    dfg_long = pd.melt(
        df, id_vars=["chunk_id"], value_vars=["node_1", "node_2"], value_name="node"
    )
    dfg_long.drop(columns=["variable"], inplace=True)
    # Self join with chunk id as the key will create a link between terms occuring in the same text chunk.
    dfg_wide = pd.merge(dfg_long, dfg_long, on="chunk_id", suffixes=("_1", "_2"))
    # drop self loops
    self_loops_drop = dfg_wide[dfg_wide["node_1"] == dfg_wide["node_2"]].index
    dfg2 = dfg_wide.drop(index=self_loops_drop).reset_index(drop=True)
    ## Group and count edges.
    dfg2 = (
        dfg2.groupby(["node_1", "node_2"])
        .agg({"chunk_id": [",".join, "count"]})
        .reset_index()
    )
    dfg2.columns = ["node_1", "node_2", "chunk_id", "count"]
    dfg2.replace("", np.nan, inplace=True)
    dfg2.dropna(subset=["node_1", "node_2"], inplace=True)
    # Drop edges with 1 count
    dfg2 = dfg2[dfg2["count"] != 1]
    dfg2["edge"] = "contextual proximity"
    return dfg2


dfg2 = contextual_proximity(dfg1)
dfg2.tail()

In [ ]:
import os
import pandas as pd

# Define output directory
output_directory = "./saved_data"

# Create the directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Save DataFrames
dfg1.to_csv(os.path.join(output_directory, "stakeholder_dfg1.csv"), index=False)
dfg2.to_csv(os.path.join(output_directory, "stakeholder_dfg2.csv"), index=False)

print("DataFrames saved successfully.")


### Merge both the dataframes

In [ ]:
print(dfg1.shape)
print(dfg2.shape)
print(dfg1.columns)
print(dfg2.columns)
# dfg1 = dfg1.drop(columns=['node_3'])


In [ ]:
dfg = pd.concat([dfg1, dfg2], axis=0)
dfg = (
    dfg.groupby(["node_1", "node_2"])
    .agg({
        "chunk_id": lambda x: ",".join(map(str, x)),
        "edge": lambda x: ','.join(map(str, x)),
        'count': 'sum'
    })
    .reset_index()
)



In [ ]:
dfg.dropna

In [ ]:
# dfg = pd.concat([dfg1, dfg2], axis=0)
# dfg = (
#     dfg.groupby(["node_1", "node_2"])
#     .agg({"chunk_id": ",".join, "edge": ','.join, 'count': 'sum'})
#     .reset_index()
# )
# dfg

## Calculate the NetworkX Graph

In [ ]:
nodes = pd.concat([dfg['node_1'], dfg['node_2']], axis=0).unique()
nodes.shape

In [ ]:
import networkx as nx
G = nx.Graph()

## Add nodes to the graph
for node in nodes:
    G.add_node(
        str(node)
    )

## Add edges to the graph
for index, row in dfg.iterrows():
    G.add_edge(
        str(row["node_1"]),
        str(row["node_2"]),
        title=row["edge"],
        weight=row['count']/4
    )

In [ ]:
# Centrality Calculations
degree_centrality = nx.degree_centrality(G)
closeness_centrality = nx.closeness_centrality(G)
betweenness_centrality = nx.betweenness_centrality(G, normalized=True)

# Add centrality measures as node attributes
for node in G.nodes():
    G.nodes[node]['degree_centrality'] = degree_centrality[node]
    G.nodes[node]['closeness_centrality'] = closeness_centrality[node]
    G.nodes[node]['betweenness_centrality'] = betweenness_centrality[node]


### Calculate communities for coloring the nodes

In [ ]:
communities_generator = nx.community.girvan_newman(G)
top_level_communities = next(communities_generator)
next_level_communities = next(communities_generator)
communities = sorted(map(sorted, next_level_communities))
print("Number of Communities = ", len(communities))
print(communities)

### Create a dataframe for community colors

In [ ]:
import seaborn as sns
palette = "hls"

## Now add these colors to communities and make another dataframe
def colors2Community(communities) -> pd.DataFrame:
    ## Define a color palette
    p = sns.color_palette(palette, len(communities)).as_hex()
    random.shuffle(p)
    rows = []
    group = 0
    for community in communities:
        color = p.pop()
        group += 1
        for node in community:
            rows += [{"node": node, "color": color, "group": group}]
    df_colors = pd.DataFrame(rows)
    return df_colors


colors = colors2Community(communities)
colors

### Add colors to the graph

In [ ]:
for index, row in colors.iterrows():
    G.nodes[row['node']]['group'] = row['group']
    G.nodes[row['node']]['color'] = row['color']
    G.nodes[row['node']]['size'] = G.degree[row['node']]

In [ ]:


# # Add nodes with centrality info to Pyvis
# for node in G.nodes():
#     title = f"""
#     Node: {node}<br>
#     Degree Centrality: {G.nodes[node]['degree_centrality']:.4f}<br>
#     Closeness Centrality: {G.nodes[node]['closeness_centrality']:.4f}<br>
#     Betweenness Centrality: {G.nodes[node]['betweenness_centrality']:.4f}
#     """
#     net.add_node(
#         node, 
#         label=str(node),
#         title=title,
#         group=G.nodes[node]['group'],
#         color=G.nodes[node]['color'],
#         size=G.nodes[node]['betweenness_centrality'] * 100  # Scale size by betweenness centrality
#     )

# # Add edges
# for index, row in dfg.iterrows():
#     net.add_edge(
#         str(row["node_1"]),
#         str(row["node_2"]),
#         title=row["edge"],
#         weight=row['count']/4
#     )

# Save the network as HTML
graph_output_directory = "./docs/stakeholder_index.html"

def save_network_with_utf8(net, path):
    html_content = net.generate_html()
    with open(path, "w", encoding="utf-8") as f:
        f.write(html_content)

net.show_buttons(filter_=["physics"])
save_network_with_utf8(net, graph_output_directory)
print(f"Graph saved to {graph_output_directory}")


In [24]:
pip install pyvis


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



   ---------------------------------------- 0.0/756.0 kB ? eta -:--:--
   - ------------------------------------- 30.7/756.0 kB 660.6 kB/s eta 0:00:02
   - ------------------------------------- 30.7/756.0 kB 660.6 kB/s eta 0:00:02
   --- ----------------------------------- 61.4/756.0 kB 409.6 kB/s eta 0:00:02
   ----- -------------------------------- 112.6/756.0 kB 656.4 kB/s eta 0:00:01
   ------- ------------------------------ 153.6/756.0 kB 654.6 kB/s eta 0:00:01
   ----------- -------------------------- 225.3/756.0 kB 808.4 kB/s eta 0:00:01
   -------------- ----------------------- 286.7/756.0 kB 886.2 kB/s eta 0:00:01
   --------------- ---------------------- 317.4/756.0 kB 893.0 kB/s eta 0:00:01
   ------------------ ------------------- 368.6/756.0 kB 916.6 kB/s eta 0:00:01
   --------------------- ---------------- 419.8/756.0 kB 937.3 kB/s eta 0:00:01
   ------------------------ ------------- 481.3/756.0 kB 942.1 kB/s eta 0:00:01
   -------------------------- ----------- 522.2/

In [6]:
import os
import pandas as pd
import networkx as nx
from pyvis.network import Network
import seaborn as sns
import random

def contextual_proximity(df: pd.DataFrame) -> pd.DataFrame:
    ## Melt the dataframe into a list of nodes
    dfg_long = pd.melt(
        df, id_vars=["chunk_id"], value_vars=["node_1", "node_2"], value_name="node"
    )
    dfg_long.drop(columns=["variable"], inplace=True)
    # Self join with chunk id as the key will create a link between terms occuring in the same text chunk.
    dfg_wide = pd.merge(dfg_long, dfg_long, on="chunk_id", suffixes=("_1", "_2"))
    # drop self loops
    self_loops_drop = dfg_wide[dfg_wide["node_1"] == dfg_wide["node_2"]].index
    dfg2 = dfg_wide.drop(index=self_loops_drop).reset_index(drop=True)
    ## Group and count edges.
    dfg2 = (
        dfg2.groupby(["node_1", "node_2"])
        .agg({"chunk_id": [",".join, "count"]})
        .reset_index()
    )
    dfg2.columns = ["node_1", "node_2", "chunk_id", "count"]
    dfg2.replace("", np.nan, inplace=True)
    dfg2.dropna(subset=["node_1", "node_2"], inplace=True)
    # Drop edges with 1 count
    dfg2 = dfg2[dfg2["count"] != 1]
    dfg2["edge"] = "contextual proximity"
    return dfg2


dfg2 = contextual_proximity(dfg1)
dfg2.tail()

# Save DataFrames
output_directory = "./saved_data"
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

dfg1.to_csv(os.path.join(output_directory, "dfg1.csv"), index=False)
dfg2.to_csv(os.path.join(output_directory, "dfg2.csv"), index=False)

print("DataFrames saved successfully.")

# Combine dfg1 and dfg2
dfg = pd.concat([dfg1, dfg2], axis=0)
dfg = (
    dfg.groupby(["node_1", "node_2"])
    .agg({
        "chunk_id": lambda x: ",".join(map(str, x)),
        "edge": lambda x: ','.join(map(str, x)),
        'count': 'sum'
    })
    .reset_index()
)

nodes = pd.concat([dfg['node_1'], dfg['node_2']], axis=0).unique()
nodes.shape

# Create a NetworkX graph
G = nx.Graph()

# Add nodes to the graph
for node in nodes:
    G.add_node(str(node))

# Add edges to the graph
for index, row in dfg.iterrows():
    G.add_edge(
        str(row["node_1"]),
        str(row["node_2"]),
        title=row["edge"],
        weight=row['count']/4
    )

# Centrality Calculations
degree_centrality = nx.degree_centrality(G)
closeness_centrality = nx.closeness_centrality(G)
betweenness_centrality = nx.betweenness_centrality(G, normalized=True)

# Add centrality measures as node attributes
for node in G.nodes():
    G.nodes[node]['degree_centrality'] = degree_centrality[node]
    G.nodes[node]['closeness_centrality'] = closeness_centrality[node]
    G.nodes[node]['betweenness_centrality'] = betweenness_centrality[node]

# Community detection
communities_generator = nx.community.girvan_newman(G)
top_level_communities = next(communities_generator)
next_level_communities = next(communities_generator)
communities = sorted(map(sorted, next_level_communities))
print("Number of Communities = ", len(communities))
print(communities)

# Color communities
def colors2Community(communities) -> pd.DataFrame:
    palette = "hls"
    p = sns.color_palette(palette, len(communities)).as_hex()
    random.shuffle(p)
    rows = []
    group = 0
    for community in communities:
        color = p.pop()
        group += 1
        for node in community:
            rows += [{"node": node, "color": color, "group": group}]
    df_colors = pd.DataFrame(rows)
    return df_colors

colors = colors2Community(communities)

for index, row in colors.iterrows():
    G.nodes[row['node']]['group'] = row['group']
    G.nodes[row['node']]['color'] = row['color']
    G.nodes[row['node']]['size'] = G.degree[row['node']]

# Create a Pyvis Network graph
net = Network(
    notebook=False,
    height="900px",
    width="100%",
    select_menu=True,
    filter_menu=False
)

# Add nodes with centrality info to Pyvis
for node in G.nodes():
    title = f"""
    Node: {node}<br>
    Degree Centrality: {G.nodes[node]['degree_centrality']:.4f}<br>
    Closeness Centrality: {G.nodes[node]['closeness_centrality']:.4f}<br>
    Betweenness Centrality: {G.nodes[node]['betweenness_centrality']:.4f}
    """
    net.add_node(
        node, 
        label=str(node),
        title=title,
        group=G.nodes[node]['group'],
        color=G.nodes[node]['color'],
        size=G.nodes[node]['betweenness_centrality'] * 100  # Scale size by betweenness centrality
    )

# Add edges
for index, row in dfg.iterrows():
    net.add_edge(
        str(row["node_1"]),
        str(row["node_2"]),
        title=row["edge"],
        weight=row['count']/4
    )

# Save the network as HTML
graph_output_directory = "./docs/index.html"

def save_network_with_utf8(net, path):
    html_content = net.generate_html()
    with open(path, "w", encoding="utf-8") as f:
        f.write(html_content)

net.show_buttons(filter_=["physics"])
save_network_with_utf8(net, graph_output_directory)
print(f"Graph saved to {graph_output_directory}")


DataFrames saved successfully.
Number of Communities =  9
[['advanced heavy water reactors (ahwrs)', 'fast reactors', 'high-temperature gas-cooled reactors (htgrs)', 'integral pressurised water reactor (ipwr)', 'liquid metal-cooled reactors', 'molten salt reactors (msrs)', 'small modular reactors (smr)', 'traveling wave reactors (twrs)', 'very high-temperature reactors (vhtrs)'], ['asean community'], ['best bet', 'cost-effectiveness', 'educating its youth about nuclear energy', 'energy independence', 'fluctuations of the fossil fuel market', 'government', 'historical events like the bombing of nagasaki', 'integrating discussions about nuclear power within the broader context of environmental sustainability and clean energy in school syllabuses', 'nuclear power', 'nuclear technologies', 'safety and thermal pollution', "singapore's energy mix", 'singaporean government'], ['city-state', 'country statuses'], ['electricity consumption', 'gross domestic product (gdp)'], ['energy 2050 committ

In [37]:
import os
import pandas as pd
import networkx as nx
from pyvis.network import Network

# Define output directory for HTML visualizations
output_directory = "./saved_data/visualizations"
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

def save_visualization(net, filename):
    filepath = os.path.join(output_directory, filename)
    net.save_graph(filepath)
    print(f"Graph saved: {filepath}")

# 1. Visualize Convergence/Divergence (Research Question 1)
def visualize_convergence_divergence(G):
    net = Network(height="900px", width="100%", select_menu=True)
    
    # Only include nodes for sentiment, stakeholders, and policy language
    for node in G.nodes():
        if 'public_sentiment' in G.nodes[node]['category'] or \
           'stakeholder' in G.nodes[node]['category'] or \
           'policy' in G.nodes[node]['category']:
            net.add_node(
                node, 
                label=node, 
                title=G.nodes[node]['category'],
                size=G.degree[node] * 10  # Scale size based on degree
            )
    
    # Add edges between relevant nodes
    for u, v, data in G.edges(data=True):
        if u in net.node_ids and v in net.node_ids:
            net.add_edge(u, v, title=data['title'], weight=data['weight'])
    
    save_visualization(net, "convergence_divergence.html")

# 2. Visualize Similarities and Differences in Policies (Research Question 2)
def visualize_policy_similarities(G, policy_countries):
    net = Network(height="900px", width="100%", select_menu=True)

    # Include nodes representing policies by country
    for country, policies in policy_countries.items():
        for policy in policies:
            net.add_node(policy, label=policy, group=country, size=G.degree[policy] * 10)
    
    # Add edges for public opinion and stakeholder connections
    for policy in policy_countries['Australia'] + policy_countries['France'] + policy_countries['Singapore']:
        for neighbor in G.neighbors(policy):
            if neighbor in policy_countries['Australia'] + policy_countries['France'] + policy_countries['Singapore']:
                net.add_edge(policy, neighbor, title=G.edges[policy, neighbor]['title'], weight=G.edges[policy, neighbor]['weight'])
    
    save_visualization(net, "policy_similarities.html")

# 3. Visualize Most Influential Policies (Research Question 3)
def visualize_influential_policies(G, most_influential_policies):
    net = Network(height="900px", width="100%", select_menu=True)

    # Include nodes for most influential policies
    for policy in most_influential_policies:
        net.add_node(policy, label=policy, size=G.degree[policy] * 20)  # Make influential policies larger
    
    # Add edges for their connections to public opinion and stakeholders
    for policy in most_influential_policies:
        for neighbor in G.neighbors(policy):
            net.add_edge(policy, neighbor, title=G.edges[policy, neighbor]['title'], weight=G.edges[policy, neighbor]['weight'])
    
    save_visualization(net, "influential_policies.html")

# 4. Visualize International Collaboration (Research Question 4)
def visualize_international_collaboration(G, collaborative_policies):
    net = Network(height="900px", width="100%", select_menu=True)

    # Include collaborative policies
    for policy in collaborative_policies:
        net.add_node(policy, label=policy, size=G.degree[policy] * 15)
    
    # Add edges for public opinion and stakeholder connections
    for policy in collaborative_policies:
        for neighbor in G.neighbors(policy):
            net.add_edge(policy, neighbor, title=G.edges[policy, neighbor]['title'], weight=G.edges[policy, neighbor]['weight'])
    
    save_visualization(net, "international_collaboration.html")


# Assuming your network graph 'G' is already built
# Assuming policy_countries is a dict categorizing policies by country as per Question 2
# Assuming most_influential_policies is a list of policies as per Question 3
# Assuming collaborative_policies is a list of policies representing international collaboration

# Call each visualization function:
visualize_convergence_divergence(G)  # Question 1
visualize_policy_similarities(G, policy_countries)  # Question 2
visualize_influential_policies(G, most_influential_policies)  # Question 3
visualize_international_collaboration(G, collaborative_policies)  # Question 4


KeyError: 'category'